# Mesogens with NP | Pressure Ramps

## Pressure: 0.0 -> 0.5

### Date: 11/13/2019 | System P = 1.8, Expected value of $T_c$ : 7.09 

In [1]:
#-----Importa los paquetes esenciales para correr la simulación.

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables que más se repitan, o sean las más importantes.

box         = 12.5;
replicates  = 10;
np_diameter = 5.0;
t_max       = 6.0;
p0 = 0.0;
p1 = 0.5;

#-------Define la celda unitaria.

hoomd.context.initialize("");
uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----De la lattice obten un snapshot e inicia el sistema desde ahí.

snap = uc.get_snapshot()
snap.replicate(replicates,replicates,replicates)
system = hoomd.init.read_snapshot(snap)

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles


In [2]:
#-----Nanopartículas

system.particles.types.add('NP');

    #-----Añade las nanopartículas
    
for nano in range(0,4):
    system.particles.add("NP")
    
    #-----Asigna a una variable las nanopartículas para despues definir sobre éstas sus propiedades.

np_0 = system.particles[1000];
np_1 = system.particles[1001];
np_2 = system.particles[1002];
np_3 = system.particles[1003];

    #-----Especifica el diametro.

np_0.diameter = np_diameter;
np_1.diameter = np_diameter;
np_2.diameter = np_diameter;
np_3.diameter = np_diameter;
    
    #------Declara la posición.

np_0.position = (0,   12.5, 0);
np_1.position = (0,  -12.5, 0);
np_2.position = (0,  37.5, 0);
np_3.position = (0, -37.5, 0);
    
    #------Declara el momento de inercia.
    
np_0.moment_inertia = [0,0,0];
np_1.moment_inertia = [0,0,0];
np_2.moment_inertia = [0,0,0];
np_3.moment_inertia = [0,0,0];

In [3]:
#-----Mesógenos

#-----Añande las partículas constituyendes de los mesógenos

system.particles.types.add('A');

    #-----Define cada mesógeno como un cuerpo rígido en un marco de referencia local a cada partícula.
    
rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

    #-----Declara el comportamiento de las partículas como el de un cuerpo rígido.

rigid.create_bodies();

#-----Define cómo se calculará la lista de vecinos y el potencial de interacción.

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Define la interacción entre las partículas de la mezcla.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0)
lj.pair_coeff.set('M' , 'M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A' , 'A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M' , 'A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('NP', 'M', epsilon = 1.0, sigma = 3.0)
lj.pair_coeff.set('NP', 'A', epsilon = 1.0, sigma = 3.0)

#-----Define el integrador y el grupo sobre el cuál se integrarán las ecuaciones de movimiento.

hoomd.md.integrate.mode_standard(dt = 0.005);

    #------Define los grupos y la unión entre ellos.

nanoparticles = hoomd.group.type(name ='Nano_Particles', type = 'NP')
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens)


pressure = hoomd.variant.linear_interp(points = [(0,p0), (1e5, p1)])
npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = t_max, tau = 1.0, tauP = 1.0, P = pressure);
npt.randomize_velocities(seed = 42)

#-----Define los archivos en los que se guardará la información de la simulación. 

log_file = "T_" + str(t_max) + "_P_" + str(p1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p1) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_max) + "_P_" + str(p1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for molecular simulation
  of colloidal systems using graphics processing units", Computer Physics
  Communications 203 (2016) 45--52
-----
notice(2): Group "Nano_Particles" created containing 4 particles
notice(2): Group "rigid_center" created containing 1000 particles
notice(2): Group "NP_Mes" created containing 1004 particles


In [4]:
#-----Run the simulation.

hoomd.run(1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9004
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1191 / 100000 | TPS 119.062 | ETA 00:13:49
Time 00:00:20 | Step 2515 / 100000 | TPS 132.291 | ETA 00:12:16
Time 00:00:30 | Step 3676 / 100000 | TPS 115.926 | ETA 00:13:50
Time 00:00:40 | Step 4824 / 100000 | TPS 114.782 | ETA 00:13:49
Time 00:00:50 | Step 5882 / 100000 | TPS 105.569 | ETA 00:14:51
Time 00:01:00 | Step 6865 / 100000 | TPS 98.2919 | ETA 00:15:47
Time 00:01:10 | Step 7830 / 100000 | TPS 96.4839 | ETA 00:15:55
Time 00:01:20 | Step 8737 / 100000 | TPS 90.5448 | ETA 00:16:47
Time 00:01:30 | Step 9615 / 100000 | TPS 87.6952 | ETA 00:17:10
Time 00:01:40 | Step 10443 / 100000 | TPS 82.6724 | ETA 00:18:03
Time 00:01:50 | Step 11190 / 100000 | TPS 74.6023 | ETA 00:19:50
Time 00:02:00 | Step 11949 / 100000 | TPS 75.7658 |

In [5]:
#-----Get density.

system.get_metadata()

OrderedDict([('box', <hoomd.data.boxdim at 0x1081db6a0>),
             ('particles', <hoomd.data.particle_data at 0x10f07c278>),
             ('number_density', 0.30870733222351987),
             ('bonds', <hoomd.data.bond_data at 0x10f07c320>),
             ('angles', <hoomd.data.angle_data at 0x10f07c390>),
             ('dihedrals', <hoomd.data.dihedral_data at 0x10f07c400>),
             ('impropers', <hoomd.data.dihedral_data at 0x10f07c470>),
             ('constraints', <hoomd.data.constraint_data at 0x10f07c4e0>),
             ('pairs', <hoomd.data.bond_data at 0x10f07c550>),
             ('timestep', 100000)])

In [6]:
#-----Get volume.

system.box.get_volume()

29166.783746751582